In [1]:
import os, sys, re
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

# EDA библиотеки
from ydata_profiling import ProfileReport
import sweetviz as sv
import dtale
import statistics as stcs
import statsmodels.stats.proportion as stsmdls

# библиотека для расчетов корреляций
from scipy import stats
from sklearn.metrics import matthews_corrcoef

# для нормализации, стандартизации
from sklearn import preprocessing

# импортируем библиотеки для разделения датасета
from sklearn.model_selection import train_test_split

# импортируем библиотеки модели для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor

# импортируем библиотеки для оценки точности модели
from sklearn import metrics

# библиотека для кодирования признаков
import category_encoders as ce

# для обработки timestamp в формате utc
import datetime, time

# работа с json
import json
from pprint import pprint

# веб-скрэпинг сайтов
import requests
from bs4 import BeautifulSoup

# выводить все результаты вычислений в ячейках Code
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# подключить Latex
from IPython.display import Latex

# загрузка файлов
import wget
import kaggle

# логгирование / логирование
import logging

# изменить формат вывода pandas
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
# pd.set_option('display.max_info_rows', 500)
# pd.set_option('display.max_info_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.large_repr', 'truncate')

In [2]:
df = pd.read_csv('./data/hotels.csv')
hotels = df.copy()
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


Задание 2.1<br>
В каких столбцах данные содержат пропущенные значения?

In [3]:
cols_null = hotels.isnull().sum()
cols_null[cols_null > 0]

lat    2448
lng    2448
dtype: int64

Задание 2.2<br>
В каких столбцах данные хранятся в числовом формате?

In [4]:
for col in hotels.columns[hotels.dtypes != 'object']:
    print(col)

additional_number_of_scoring
average_score
review_total_negative_word_counts
total_number_of_reviews
review_total_positive_word_counts
total_number_of_reviews_reviewer_has_given
reviewer_score
lat
lng


Задание 2.3<br>
В каких столбцах данные представляют собой строки?

!= review_date (exclude)

In [5]:
for col in hotels.columns[hotels.dtypes == 'object']:
    print(col)

hotel_address
review_date
hotel_name
reviewer_nationality
negative_review
positive_review
tags
days_since_review


Задание 2.4<br>
Напишите признак, представляющий собой дату.

* review_date (True)

Задание 3.1<br>
Вопрос для размышления. По какой причине во время обучения модели возникла ошибка?

* Модель не может работать с параметрами по умолчанию. Параметры нужно настраивать вручную, а мы этого не делали.
* Проблема в названиях столбцов: они слишком длинные, и там используются не только буквы, но и знаки подчёркивания.
* Ошибка возникла при обработке столбца со строковыми значениями. Следовательно, для обучения модели все данные должны быть переданы только в числовом формате. (True)

Задание 4.2<br>
Сколько уникальных названий отелей представлено в наборе данных?

In [6]:
hotels['hotel_name'].nunique()

1492

In [7]:
hotels['review_date'].head()

0     2/19/2016
1     1/12/2017
2    10/18/2016
3     9/22/2015
4      3/5/2016
Name: review_date, dtype: object

Задание 4.3<br>
Ответьте на вопросы о дате размещения отзывов.

In [8]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'], format='%m/%d/%Y')

In [9]:
hotels['review_date'].head()

0   2016-02-19
1   2017-01-12
2   2016-10-18
3   2015-09-22
4   2016-03-05
Name: review_date, dtype: datetime64[ns]

Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd:

In [10]:
hotels['review_date'].max()

Timestamp('2017-08-03 00:00:00')

Когда был оставлен самый первый отзыв? Введите ответ в формате yyyy-mm-dd:

In [11]:
hotels['review_date'].min()

Timestamp('2015-08-04 00:00:00')

Задание 4.4<br>
Сколько уникальных тегов представлено в наборе данных?

In [12]:
hotels['tags'][0]

"[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']"

In [13]:
# Сколько уникальных тегов представлено в наборе данных?
tags = {}
for tag_list in hotels['tags']:
    for _tag in tag_list[3:-3].split(" ', ' "):
        if _tag not in tags:
            tags[_tag] = 1
        else:
            tags[_tag] += 1

# вывести все теги
# print(tags)

# вывести количество тегов
print(len(tags))

2368


Задание 4.5<br>
Какой тег представлен в наибольшем числе отзывов?

In [14]:
# счётчик
tag_cnt = 0

# переменная-накопитель - самый частый тег
tag_name = ''
for _tag in tags:
    # если встретили тег "пожирнее", переписываем соответствующие переменные:
    # сам тег (ключ словаря), и значение ключа (значение ключа)
    if tags[_tag] > tag_cnt:
        tag_name = _tag
        tag_cnt = tags[_tag]
print(tag_name, tag_cnt)

Leisure trip 313593


Задание 4.6<br>
Из тегов выясните, на сколько ночей чаще всего останавливаются путешественники в отелях.

In [15]:

for _tag in tags:
    if _tag.lower().find('night') >= 1:
    # if a.lower().find('night') > -1:
        print(_tag, tags[_tag])

Stayed 2 nights 100263
Stayed 1 night 145373
Stayed 3 nights 72000
Stayed 6 nights 7399
Stayed 4 nights 35748
Stayed 5 nights 15611
Stayed 8 nights 1910
Stayed 7 nights 5549
Stayed 10 nights 663
Stayed 14 nights 184
Stayed 19 nights 23
Stayed 13 nights 174
Stayed 9 nights 966
Stayed 17 nights 27
Stayed 11 nights 306
Stayed 27 nights 10
Stayed 12 nights 217
Stayed 15 nights 87
Stayed 21 nights 19
Stayed 18 nights 24
Stayed 16 nights 38
Stayed 26 nights 6
Special Offer Double Room 3 nights minimum 6
Stayed 24 nights 5
Stayed 30 nights 10
Stayed 22 nights 8
King Room with Knightsbridge View 8
Suite with Knightsbridge View 1
Stayed 23 nights 6
Stayed 28 nights 7
Stayed 20 nights 17
Junior Suite Give me the night 4
Stayed 25 nights 4
Knightsbridge Suite 1
Stayed 29 nights 3


Задание 3.2<br>
Какое значение метрики MAPE вам удалось получить на этом этапе?
(Перенесено в конец, т.к. удаляются (пока ещё нужные для исследования) столбцы)

In [16]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" или "datetime64[ns]"" обучаться не будет, просто выберим их и удалим
object_columns = [col for col in hotels.columns 
    if ((hotels[col].dtypes == 'object') | 
    (hotels[col].dtypes == 'datetime64[ns]'))
]
hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

In [17]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [18]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  

# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

RandomForestRegressor()

In [20]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14142975122323376
